In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
394777,357,357,315,85.763716,216.152455,165.0,9.064458,13.248054,5.000000,-201.300003,-173.000000,-332.750000,0.697266,0.697266,5.000000,1
684578,359,359,300,176.695312,130.417969,85.0,3.505859,4.908203,5.000000,-172.149414,-319.149414,-687.200012,0.701172,0.701172,5.000000,32
696481,387,387,235,157.974609,95.994141,60.0,14.361328,15.873047,5.000000,-196.027267,-193.122070,-419.720825,0.755859,0.755859,5.000000,5
649546,349,349,294,146.552734,82.137695,175.0,22.494141,16.359375,4.986448,-175.159180,-282.159180,-234.014008,0.681641,0.681641,4.986448,5
385457,360,360,245,148.359375,218.320312,192.0,33.046875,18.281250,20.000000,-166.000000,-180.000000,-240.000000,0.703125,0.703125,5.000000,5
655480,315,315,285,179.340820,199.951172,245.0,3.691406,4.306641,5.000000,-161.192383,-297.192383,-580.799988,0.615234,0.615234,5.000000,1
696788,303,303,300,176.947266,182.273438,285.0,10.060547,10.060547,5.000000,-149.204102,-291.204102,-694.700012,0.591797,0.591797,5.000000,5
674088,333,333,205,137.882812,112.517578,70.0,7.154297,14.958984,5.000000,-180.174805,-306.174805,-547.200012,0.650391,0.650391,5.000000,5
699552,350,350,270,76.562500,129.541016,110.0,4.785156,5.468750,5.000000,-165.942673,-174.658203,360.936890,0.683594,0.683594,5.000000,1


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_train.index))
print('distinct lungs:', len(lungs))

distinct lungs: 1470


In [6]:
df_tasks = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train/tasks_lung.csv', index_col=['seriesuid'])
df_tasks.index = df_tasks.index.astype('str')

print('tasks:', df_tasks.shape, 'distinct lung:', len(set(df_tasks.index)))

tasks: (5872, 7) distinct lung: 100


# 3. Predict Results by Task_test.csv

In [7]:
df_tasks_nms = df_tasks

# df_tasks_nms = pd.DataFrame(list_tasks_nms, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
# df_tasks_nms.index = df_tasks_nms.index.astype('str')

print('tasks:', df_tasks_nms.shape, 'distinct lung:', len(set(df_tasks_nms.index)))

tasks: (5872, 7) distinct lung: 100


In [8]:
df_tasks_nms.head(10)

,vcoordX,vcoordY,vcoordZ,diametersX,diametersY,diametersZ,probability
seriesuid,,,,,,,
671752,215,94,31,32,37,19,0.999581
671752,123,119,106,8,7,6,0.998630
671752,91,131,129,9,9,5,0.998667
671752,126,128,126,5,6,5,0.997698
671752,101,152,130,8,7,5,0.999174
671752,232,192,158,5,4,3,0.974926
671752,64,123,183,7,6,5,0.997812
671752,199,162,185,12,12,10,0.998443
671752,121,101,215,6,7,5,0.999822


In [9]:
import tensorflow.keras.backend as K
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [10]:
from tensorflow.keras.models import load_model
output_dir = './output/densenet3d/1564559396/' # labels in center coord
model = load_model(output_dir + "model-best.hd5")

W0731 23:53:19.756293 140345980491584 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 23:53:19.757599 140345980491584 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 23:53:19.764877 140345980491584 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [11]:
labels = [0, 1, 5]
batch_size = 32
threshold_probability = 0.7
wtype = 'lung'
plot = False

In [12]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [13]:
list_results = []
print(wtype, 'window')
count_lung = 0
for uid in set(df_tasks_nms.index):
    tasks = df_tasks_nms.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks))
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    # convert coordX to vcoordX
#     tasks['vcoordX'] = tasks['coordX'] - meta.originX
#     tasks['vcoordY'] = tasks['coordY'] - meta.originY
#     tasks['vcoordZ'] = tasks['coordZ'] - meta.originZ

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []
            
        flag_range = True
        if (item.vcoordX < 40) or (item.vcoordX > 360):
            flag_range = False
        if (item.vcoordY < 40) or (item.vcoordY > 300):
            flag_range = False
        if item.vcoordZ > 350:
            flag_range = False  

        # patch without mask
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=32)

        if np.sum(cube) > 0 and flag_range:
            X_item.append(item)
            X_test.append(cube)
        # else:
            # print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size*5, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                if np.argmax(predictions_test[i]) > 0:
                    category = int(np.argmax(predictions_test[i]))
                    probability = float(np.max(predictions_test[i]))

                    result = {}
                    result['seriesuid'] = uid
                    result['coordX'] = itm.vcoordX + meta.originX
                    result['coordY'] = itm.vcoordY + meta.originY
                    result['coordZ'] = itm.vcoordZ + meta.originZ
                    result['class'] = helper.get_origin_class_by_predicted_value(category, labels)
                    result['probability'] = probability
                    result['wtype'] = wtype

                    if probability > threshold_probability:
                        print(result)
                        if plot:
                            plot_cube((cube*255).astype(np.uint8), rows=4, cols=8)
                        list_results.append(result)

        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','coordX','coordY','coordZ','class','probability','wtype'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')
    
    count_lung += 1

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)), '/', count_lung)

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype +'.csv', encoding='utf-8')

    print('-'*100)
    

lung window
lung: 645337 tasks: 49
48/48 [==============================] - 3s 69ms/sample
{'seriesuid': '645337', 'coordX': 29.268478393554688, 'coordY': 26.341796875, 'coordZ': -253.922119140625, 'class': 5, 'probability': 0.901622474193573, 'wtype': 'lung'}
{'seriesuid': '645337', 'coordX': 1.2684783935546875, 'coordY': 44.341796875, 'coordZ': -170.922119140625, 'class': 1, 'probability': 0.9299537539482117, 'wtype': 'lung'}
{'seriesuid': '645337', 'coordX': 3.2684783935546875, 'coordY': -59.658203125, 'coordZ': -127.922119140625, 'class': 1, 'probability': 0.9379937052726746, 'wtype': 'lung'}
{'seriesuid': '645337', 'coordX': 90.26847839355469, 'coordY': 20.341796875, 'coordZ': -120.922119140625, 'class': 5, 'probability': 0.8723357915878296, 'wtype': 'lung'}
{'seriesuid': '645337', 'coordX': -143.7315216064453, 'coordY': -18.658203125, 'coordZ': -296.922119140625, 'class': 5, 'probability': 0.7034125328063965, 'wtype': 'lung'}
{'seriesuid': '645337', 'coordX': 31.268478393554688, 

44/44 [==============================] - 0s 3ms/sample
{'seriesuid': '663672', 'coordX': -63.31117248535156, 'coordY': -85.658203125, 'coordZ': -81.3720703125, 'class': 5, 'probability': 0.9822059869766235, 'wtype': 'lung'}
{'seriesuid': '663672', 'coordX': 101.68882751464844, 'coordY': -22.658203125, 'coordZ': -8.3720703125, 'class': 1, 'probability': 0.7957313060760498, 'wtype': 'lung'}
{'seriesuid': '663672', 'coordX': -116.31117248535156, 'coordY': 20.341796875, 'coordZ': -74.3720703125, 'class': 1, 'probability': 0.7750519514083862, 'wtype': 'lung'}
{'seriesuid': '663672', 'coordX': -57.31117248535156, 'coordY': -46.658203125, 'coordZ': -49.3720703125, 'class': 5, 'probability': 0.8887553215026855, 'wtype': 'lung'}
{'seriesuid': '663672', 'coordX': -27.311172485351562, 'coordY': -60.658203125, 'coordZ': -18.3720703125, 'class': 5, 'probability': 0.9205266833305359, 'wtype': 'lung'}
{'seriesuid': '663672', 'coordX': -77.31117248535156, 'coordY': -48.658203125, 'coordZ': 21.62792968

60/60 [==============================] - 0s 3ms/sample
{'seriesuid': '656238', 'coordX': -18.352005004882812, 'coordY': 30.889694213867188, 'coordZ': -923.0, 'class': 5, 'probability': 0.806015133857727, 'wtype': 'lung'}
{'seriesuid': '656238', 'coordX': 110.64799499511719, 'coordY': -38.11030578613281, 'coordZ': -854.0, 'class': 1, 'probability': 0.9883434176445007, 'wtype': 'lung'}
{'seriesuid': '656238', 'coordX': -29.352005004882812, 'coordY': -31.110305786132812, 'coordZ': -760.0, 'class': 1, 'probability': 0.8430901765823364, 'wtype': 'lung'}
{'seriesuid': '656238', 'coordX': 148.6479949951172, 'coordY': 31.889694213867188, 'coordZ': -886.0, 'class': 1, 'probability': 0.8014170527458191, 'wtype': 'lung'}
{'seriesuid': '656238', 'coordX': -81.35200500488281, 'coordY': -60.11030578613281, 'coordZ': -842.0, 'class': 1, 'probability': 0.8110778331756592, 'wtype': 'lung'}
{'seriesuid': '656238', 'coordX': -67.35200500488281, 'coordY': -50.11030578613281, 'coordZ': -845.0, 'class': 1, 

31/31 [==============================] - 0s 4ms/sample
{'seriesuid': '695349', 'coordX': 83.8095703125, 'coordY': -78.1904296875, 'coordZ': -417.1000061035156, 'class': 1, 'probability': 0.990462601184845, 'wtype': 'lung'}
{'seriesuid': '695349', 'coordX': -100.1904296875, 'coordY': -174.1904296875, 'coordZ': -347.1000061035156, 'class': 1, 'probability': 0.9985730648040771, 'wtype': 'lung'}
{'seriesuid': '695349', 'coordX': -70.1904296875, 'coordY': -138.1904296875, 'coordZ': -286.1000061035156, 'class': 1, 'probability': 0.7154580354690552, 'wtype': 'lung'}
{'seriesuid': '695349', 'coordX': 27.8095703125, 'coordY': -65.1904296875, 'coordZ': -450.1000061035156, 'class': 5, 'probability': 0.9490789771080017, 'wtype': 'lung'}
{'seriesuid': '695349', 'coordX': 26.8095703125, 'coordY': -59.1904296875, 'coordZ': -451.1000061035156, 'class': 5, 'probability': 0.9355820417404175, 'wtype': 'lung'}
{'seriesuid': '695349', 'coordX': 69.8095703125, 'coordY': -55.1904296875, 'coordZ': -449.100006

51/51 [==============================] - 0s 3ms/sample
{'seriesuid': '655426', 'coordX': 81.8310546875, 'coordY': -78.1689453125, 'coordZ': -1020.699951171875, 'class': 1, 'probability': 0.9855815768241882, 'wtype': 'lung'}
{'seriesuid': '655426', 'coordX': -78.1689453125, 'coordY': -150.1689453125, 'coordZ': -1093.699951171875, 'class': 1, 'probability': 0.791010320186615, 'wtype': 'lung'}
{'seriesuid': '655426', 'coordX': 72.8310546875, 'coordY': -150.1689453125, 'coordZ': -1071.699951171875, 'class': 1, 'probability': 0.901651918888092, 'wtype': 'lung'}
{'seriesuid': '655426', 'coordX': 35.8310546875, 'coordY': -124.1689453125, 'coordZ': -1065.699951171875, 'class': 5, 'probability': 0.9957373142242432, 'wtype': 'lung'}
{'seriesuid': '655426', 'coordX': -82.1689453125, 'coordY': -88.1689453125, 'coordZ': -1072.699951171875, 'class': 1, 'probability': 0.9235432744026184, 'wtype': 'lung'}
{'seriesuid': '655426', 'coordX': 40.8310546875, 'coordY': -83.1689453125, 'coordZ': -1065.699951

...

26/26 [==============================] - 0s 3ms/sample
{'seriesuid': '656835', 'coordX': 79.0, 'coordY': 204.0, 'coordZ': -415.95001220703125, 'class': 1, 'probability': 0.8440650701522827, 'wtype': 'lung'}
{'seriesuid': '656835', 'coordX': -50.0, 'coordY': 89.0, 'coordZ': -379.95001220703125, 'class': 5, 'probability': 0.9859578013420105, 'wtype': 'lung'}
{'seriesuid': '656835', 'coordX': -49.0, 'coordY': 96.0, 'coordZ': -397.95001220703125, 'class': 5, 'probability': 0.9703876376152039, 'wtype': 'lung'}
{'seriesuid': '656835', 'coordX': 73.0, 'coordY': 120.0, 'coordZ': -405.95001220703125, 'class': 5, 'probability': 0.9657090902328491, 'wtype': 'lung'}
{'seriesuid': '656835', 'coordX': -49.0, 'coordY': 97.0, 'coordZ': -386.95001220703125, 'class': 5, 'probability': 0.9952900409698486, 'wtype': 'lung'}
{'seriesuid': '656835', 'coordX': -84.0, 'coordY': 139.0, 'coordZ': -315.95001220703125, 'class': 5, 'probability': 0.7292143106460571, 'wtype': 'lung'}
total: (1172, 6) lung: 82 / 82
-

49/49 [==============================] - 0s 3ms/sample
{'seriesuid': '696418', 'coordX': 38.95176696777344, 'coordY': 23.341796875, 'coordZ': 1147.1400146484377, 'class': 5, 'probability': 0.8796717524528503, 'wtype': 'lung'}
{'seriesuid': '696418', 'coordX': -59.04823303222656, 'coordY': -51.658203125, 'coordZ': 1214.1400146484377, 'class': 1, 'probability': 0.8857442140579224, 'wtype': 'lung'}
{'seriesuid': '696418', 'coordX': 99.95176696777344, 'coordY': 44.341796875, 'coordZ': 1125.1400146484377, 'class': 1, 'probability': 0.7591580152511597, 'wtype': 'lung'}
{'seriesuid': '696418', 'coordX': -36.04823303222656, 'coordY': -44.658203125, 'coordZ': 1189.1400146484377, 'class': 5, 'probability': 0.9856831431388855, 'wtype': 'lung'}
{'seriesuid': '696418', 'coordX': -29.048233032226562, 'coordY': -61.658203125, 'coordZ': 1191.1400146484377, 'class': 5, 'probability': 0.9481980800628662, 'wtype': 'lung'}
{'seriesuid': '696418', 'coordX': -90.04823303222656, 'coordY': 14.341796875, 'coor

46/46 [==============================] - 0s 3ms/sample
{'seriesuid': '689124', 'coordX': -74.89328002929688, 'coordY': -87.1669921875, 'coordZ': 1236.8861083984375, 'class': 1, 'probability': 0.9935622215270996, 'wtype': 'lung'}
{'seriesuid': '689124', 'coordX': 37.106719970703125, 'coordY': -0.1669921875, 'coordZ': 1290.8861083984375, 'class': 5, 'probability': 0.975955069065094, 'wtype': 'lung'}
{'seriesuid': '689124', 'coordX': 86.10671997070312, 'coordY': 18.8330078125, 'coordZ': 1333.8861083984375, 'class': 1, 'probability': 0.8062868118286133, 'wtype': 'lung'}
{'seriesuid': '689124', 'coordX': 47.106719970703125, 'coordY': -46.1669921875, 'coordZ': 1417.8861083984375, 'class': 1, 'probability': 0.8999004364013672, 'wtype': 'lung'}
{'seriesuid': '689124', 'coordX': 66.10671997070312, 'coordY': -48.1669921875, 'coordZ': 1482.8861083984375, 'class': 1, 'probability': 0.8014522194862366, 'wtype': 'lung'}
{'seriesuid': '689124', 'coordX': 95.10671997070312, 'coordY': 19.8330078125, 'c

88/88 [==============================] - 0s 3ms/sample
{'seriesuid': '664003', 'coordX': 132.322265625, 'coordY': -213.677734375, 'coordZ': 42.29999923706055, 'class': 1, 'probability': 0.9989019632339478, 'wtype': 'lung'}
{'seriesuid': '664003', 'coordX': 124.322265625, 'coordY': -180.677734375, 'coordZ': 115.29999923706055, 'class': 1, 'probability': 0.9645875692367554, 'wtype': 'lung'}
{'seriesuid': '664003', 'coordX': 100.322265625, 'coordY': -183.677734375, 'coordZ': -19.700000762939453, 'class': 5, 'probability': 0.7207794189453125, 'wtype': 'lung'}
{'seriesuid': '664003', 'coordX': 3.322265625, 'coordY': -112.677734375, 'coordZ': -13.700000762939453, 'class': 5, 'probability': 0.9366254806518555, 'wtype': 'lung'}
{'seriesuid': '664003', 'coordX': -69.677734375, 'coordY': -190.677734375, 'coordZ': 12.299999237060547, 'class': 5, 'probability': 0.801137387752533, 'wtype': 'lung'}
{'seriesuid': '664003', 'coordX': 187.322265625, 'coordY': -152.677734375, 'coordZ': 16.29999923706054

55/55 [==============================] - 0s 3ms/sample
{'seriesuid': '657853', 'coordX': -130.6171875, 'coordY': -90.6171875, 'coordZ': -427.9000244140625, 'class': 5, 'probability': 0.9349488615989685, 'wtype': 'lung'}
{'seriesuid': '657853', 'coordX': 17.3828125, 'coordY': -88.6171875, 'coordZ': -430.9000244140625, 'class': 1, 'probability': 0.9368531107902527, 'wtype': 'lung'}
{'seriesuid': '657853', 'coordX': 68.3828125, 'coordY': -90.6171875, 'coordZ': -430.9000244140625, 'class': 1, 'probability': 0.9955888986587524, 'wtype': 'lung'}
{'seriesuid': '657853', 'coordX': -75.6171875, 'coordY': -167.6171875, 'coordZ': -540.9000244140625, 'class': 5, 'probability': 0.9450557827949524, 'wtype': 'lung'}
{'seriesuid': '657853', 'coordX': -40.6171875, 'coordY': -181.6171875, 'coordZ': -502.9000244140625, 'class': 1, 'probability': 0.8010480999946594, 'wtype': 'lung'}
{'seriesuid': '657853', 'coordX': -141.6171875, 'coordY': -146.6171875, 'coordZ': -502.9000244140625, 'class': 1, 'probabili

In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype +'.csv', index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))

results: (1412, 6) distinct lung: 100


In [15]:
df_results = df_results[df_results['probability'] > 0.95]
df_results = df_results.drop(columns=['wtype'])
print(df_results.shape, 'distinct lung:', len(set(df_results.index)))

(627, 5) distinct lung: 97


In [16]:
df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype + '_filtered.csv', encoding='utf-8')

print('1:', len(df_results[df_results['class']==1]), '5:', len(df_results[df_results['class']==5]))

1: 229 5: 398
